# Evaluation Notebook
This notebook provided some methods to do evaluate Wikisim on different datasets. 

# Configs

In [ ]:
%%system

mysql -u root -pemilios -e 'set global key_buffer_size=4*1024*1024*1024;'
mysql -u root -pemilios -e 'set global bulk_insert_buffer_size=1024*1024*1024;'
mysql -u root -pemilios -e 'set global query_cache_size = 4*1024*1024*1024;'
mysql -u root -pemilios -e 'set global query_cache_limit = 4*1024*1024*1024;'
mysql -u root -pemilios -e 'set global tmp_table_size = 4*1024*1024*1024;'

In [5]:
%load_ext autoreload
%autoreload

import datetime
import logging
import os.path
import sys



home = os.path.expanduser("~");
dsdir = os.path.join(home ,"backup/projects/wikisim/datasets/");
workingdir = os.path.join(home , 'backup/tmp/');
baseresdir = path = os.path.join(workingdir, 'results')

logging.basicConfig(filename=os.path.join(workingdir,'myapp.log'), level=logging.INFO);    

    
def resdir(hitsver, direction=None):   
    path = os.path.join(baseresdir , graphtype(direction), hitsver);
    if not os.path.exists(path):
        os.makedirs(path)
    return path

    
def tmpdir(direction, hitsver):
    return os.path.join(resdir(hitsver, direction),'tmp/');

def graphdir(direction):    
    return os.path.join(getworkingdir , 'graphs' , wikisim.graphtypestr(direction));

def initdirs(direction, hitsver):
    path = resdir(hitsver, direction)
    if not os.path.exists(path):
        os.makedirs(path)
    path = tmpdir(direction, hitsver)
    if not os.path.exists(path):
        os.makedirs(path)
def printflush(*str):
    print str
    sys.stdout.flush()
    
    
def graphtype(direction):
    if direction == None:
        return ''
    if direction == DIR_IN:
        return 'in'
    if direction == DIR_OUT:
        return 'out'
    if direction == DIR_BOTH:
        return 'both' 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Evaluating rvspage rank

In [6]:

import os
import time;
#D = sparse(k,k,1./c(k),n,n);

# %autoreload 1

# %aimport wikipedia
# %aimport visualize
# from visualize import *
# from wikipedia import *
from calcsim import *

direction=DIR_OUT;
method = 'rvspagerank'
initdirs(direction, 'rvspagerank')
resfilename =  os.path.join(baseresdir, 'reslog.txt')

dsfiles=('MC_28-edited.csv', 'MiniMayoSRS-edited.csv',)
dsfiles=('MiniMayoSRS-edited.csv',)
start = time.time()
for dsname in dsfiles:
    printflush ("Processing",dsname)
    dsbase, dsext = os.path.splitext(dsname);
    infilename = os.path.join(dsdir, dsname)
    outfilename = os.path.join(resdir(method, direction), dsbase+ '.out'+ dsext)
    _ , corr = getsim_file(infilename, outfilename, method, direction);
    logres(resfilename, 'rvspagerank\t%s\t%s', graphtype(direction), corr)
    print corr
    
print str(timeformat(int(time.time()-start)));
#close()   

('Processing', 'MiniMayoSRS-edited.csv')
SpearmanrResult(correlation=0.68416972848512614, pvalue=4.2679377614525189e-05)
0:00:00


# Embedding

In [3]:
from scipy import stats
import os
import time;


%aimport calcsim
from calcsim import *



direction=DIR_OUT;
initdirs(direction, 'rvspagerank')

dsfiles=('MC_28-edited.csv', 'MiniMayoSRS-edited.csv',)
dsfiles=('MiniMayoSRS-edited.csv',)
start = time.time()
for dsname in dsfiles:
    printflush ("Processing",dsname)
    dsbase, dsext = os.path.splitext(dsname);
    infilename = os.path.join(dsdir, dsname)
    outfilename = os.path.join(resdir('rvspagerank', direction), dsbase+ '.emb'+ dsext)
    getembed_file(infilename, outfilename,direction, cutoff=3);
    
print str(timeformat(int(time.time()-start)));
#close()   

('Processing', 'MiniMayoSRS-edited.csv')
0:00:00


# In-Out degree distribution

In [ ]:
import operator
from collections import defaultdict
with open(home+"/backup/projects/datasets/embed/allids.csv") as f:
    allids=set(line.strip() for line in f);
#outcount = defaultdict();    
for id in allids:
    if not id.isdigit(): continue;
    outcount[id]=len(getlinkedpages(id,Wikipedia.DIR_IN));
#o=w.getlinkedpages_query('None',Wikipedia.DIR_OUT);
#print o;
#w.cursor.execute(o)
#s=w.getlinkedpages_query(29953972,Wikipedia.DIR_OUT);
outcount_sorted=sorted(outcount.items(), key=operator.itemgetter(1))
print outcount_sorted

w.close();    